# Reducir dimensionalidad con PCA (Análisis de Componentes Principales)

**Curso**: CC5213 - Recuperación de Información Multimedia  
**Profesor**: Juan Manuel Barrios  
**Fecha**: 21 de junio de 2025  


### Gráficos interactivos

Para los gráficos se usa matplotlib:
```
pip install matplotlib
```

Para gráficos interactivos (por ej. hacer zoom):

  1. Instalar ipympl:  `pip install ipympl`
  2. Reiniciar jupyter 
  3. Reemplazar `%matplotlib inline` por `%matplotlib widget`


In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

## Descomentar esta linea para graficos interactivos
## %matplotlib widget


### Leer los datos

In [ ]:
import numpy

# cargar un conjunto de vectores
dataset_q = numpy.load("dataset_a_q.npy")
dataset_r = numpy.load("dataset_a_r.npy")

print("Dataset A: conjunto_Q={} conjunto_R={}".format(dataset_q.shape, dataset_r.shape))

## Paso 1: centrar los vectores del dataset

In [ ]:
promedios = dataset_r.mean(axis=0)
print("vector_promedio: {}".format(promedios))

datos_centrados = dataset_r - promedios

## Paso 2: calcular matriz de convarianza

In [ ]:
# se usa transpose() para que calcule una matriz de 128x128, bias=False para varianzas divididas por n-1
matriz_covarianza = numpy.cov(datos_centrados.transpose(), bias=False)
print("matriz_covarianza: {}".format(matriz_covarianza.shape))
print(matriz_covarianza)

## Paso 3: calcular valores y vectores propios de la matriz de covarianza

In [ ]:
eigenvalues, eigenvectors = numpy.linalg.eig(matriz_covarianza)
print("eigenvalues: {}".format(eigenvalues.shape))
print("eigenvectors: {}".format(eigenvectors.shape))

## Paso 4: ordenar valores propios de mayor a menor (mantener el mismo orden en los vectores propios)

In [ ]:
# indices para ordenar
indices_menor_a_mayor = eigenvalues.argsort()
indices_mayor_a_menor = indices_menor_a_mayor[::-1]

# obtener valores y vectores ordenados de mayor a menor
eigenvalues = eigenvalues[indices_mayor_a_menor]
eigenvectors = eigenvectors[:, indices_mayor_a_menor]

print("valores propios={}".format(eigenvalues))

### Varianza acumulada por los primeros N valores propios

In [ ]:
def varianza_acumulada(dimension):
    return numpy.sum(eigenvalues[:dimension]) / numpy.sum(eigenvalues)


dimensiones = []
varianzas = []
for dim in range(0, 129, 4):
    pct_varianza = 100 * varianza_acumulada(dim)
    pct_dim = 100 * dim / dataset_r.shape[1]
    print("{}-d\tdim={:.1f}%\tvar={:.1f}%\t".format(dim, pct_dim, pct_varianza))
    dimensiones.append(pct_dim)
    varianzas.append(pct_varianza)

In [ ]:
plt.plot(
    dimensiones,
    varianzas,
    label="varianza",
    color="b",
    linestyle="-",
    marker="o",
    markerfacecolor="g",
    markersize=6,
)
plt.title("Varianza total al considerar un % de dimensiones")
plt.xlabel("dimensión %")
plt.ylabel("varianza acumulada %")
plt.xlim(0, 100)
plt.ylim(0, 100)
plt.legend()
plt.show()

En el gráfico anterior se ve que la mayor cantidad de varianza se encuentra en las primeras componentes principales:   
El 50% de la varianza se encuentra en las primeras 8 coordenadas.  
El 75% de la varianza se encuentra en las primeras 24 coordenadas.  
El 95% de la varianza se encuentra en las primeras 68 coordenadas.

## Paso 5: matriz de transformación

In [ ]:
# unir primeros N vectores propios
def transformar(dataset, new_dims):
    centrado = dataset - promedios
    transformacion = eigenvectors[:, :new_dims]
    transformado = centrado.dot(transformacion)
    return transformado

## Paso 6: transformar matriz de descriptores

In [ ]:
print("ORIGINALES Q={} R={}".format(dataset_q.shape, dataset_r.shape))

r_reducido = transformar(dataset_r, 10)
q_reducido = transformar(dataset_q, 10)

print("REDUCIDOS  Q={}  R={}".format(q_reducido.shape, r_reducido.shape))

## Comprobación: los vectores transformados no tienen covarianzas

In [ ]:
print("r_reducido: {}".format(r_reducido.shape))

r_reducido_centrado = r_reducido - r_reducido.mean(axis=0)
print("r_reducido_centrado: {}".format(r_reducido_centrado.shape))

r_reducido_covarianza = numpy.cov(r_reducido_centrado.transpose(), bias=False)
print("matriz_covarianza: {}".format(r_reducido_covarianza.shape))

# las covarianzas debieran ser cercanas a 0
print()
print("covarianzas={}".format(r_reducido_covarianza[0][1]))
print("covarianzas={}".format(r_reducido_covarianza[2][3]))
print("covarianzas={}".format(r_reducido_covarianza[4][3]))
print()
print("varianzas={}".format(r_reducido_covarianza[0][0]))
print("varianzas={}".format(r_reducido_covarianza[1][1]))
print("varianzas={}".format(r_reducido_covarianza[2][2]))
print()
print("matriz de covarianza={}".format(r_reducido_covarianza))

## Ejercicio: Transformar otro conjunto de vectores

In [ ]:
# probar con otro dataset
dataset_q = numpy.load("dataset_b_q.npy")
dataset_r = numpy.load("dataset_b_r.npy")

print("Dataset B: conjunto_Q={} conjunto_R={}".format(dataset_q.shape, dataset_r.shape))

# probar de nuevo el resultado con este dataset y verificar si entregs los mismos valores

**Ejercicio:** Comparar las varianzas acumuladas de este dataset y el anterior. **¿Se distribuyen igual los porcentajes de varianza entre los datasets A y B?**